In [1]:
# dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from random import randint
from time import sleep

### Scraping from Billboard

In [2]:
# determine weeks needed to scrape
# get today's date
import datetime as dt
from datetime import date, timedelta, datetime
today = datetime.today()

# get last day data was scraped from sql ** this has to be the last day data is got from,
# not necessarily the day data was last scraped
# at the moment, the day is entered manually
# on the next run of the JN, the following code can be used (csv):

# last_date_test = pd.read_csv('../data/scrapeRecord.csv')
# date_time_str = last_date_test.iloc[:, 0][0]

date_time_str = '2021-05-01'

# convert that day string into date
last_update = dt.datetime.strptime(date_time_str,"%Y-%m-%d") 


# find number of days since scraping occurred
num_days_since = int(str(today - last_update).split(' ')[0])
num_weeks_since = round(num_days_since/7)
date_to_start = str(last_update + dt.timedelta(days=7)).split(' ')[0]
print(f'days: {num_days_since}, in weeks: {num_weeks_since}')
print(date_to_start)

days: 13, in weeks: 2
2021-05-08


In [3]:
# set up url based on last date scraped
billboard_url = f'https://www.billboard.com/charts/Hot-100/{date_to_start}'
print(billboard_url)

https://www.billboard.com/charts/Hot-100/2021-05-08


In [4]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/sarahdalleyhood/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [5]:
# visit the site
browser.visit(billboard_url)
#sleep(randint(3,10))

In [6]:
# scraping billboard song info
counter = 0
billboard_info_list = []
# going for range of weeks inbetween last scraped day and present day
for counter in range(0, num_weeks_since):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    date_section = soup.find('div', class_='chart-detail-header__select-date')
    span_section = date_section.find('span')
    button = span_section.find('button')
    this_date = button.text
    print(this_date)
    
    for item in soup.find_all('div', class_='chart-details__left-rail'):
        list_item = item.find_all('div', class_='chart-list-item')
        for s in list_item:
            # current place on charts
            list_rank = s.get('data-rank')
            # performer
            list_artist = s.get('data-artist')
            # song name
            list_title = s.get('data-title')
            # getting previous stats
            list_extra = s.find('div', class_='chart-list-item__first-row')  
            ministats = list_extra.find_all('div', class_='chart-list-item__ministats-cell')
            for c in ministats:
                # last week's place on charts
                if c.find('span', class_='chart-list-item__ministats-cell-heading').text == 'Last':
                    lastWeek = c.text
                    lastWeekCleaned = lastWeek.strip().split('L')[0].strip()
                # peak of the song
                if c.find('span', class_='chart-list-item__ministats-cell-heading').text == 'Peak':
                    songPeak = c.text
                    songPeakCleaned = songPeak.strip().split('P')[0].strip()
                # how long the song has been on the chart
                if c.find('span', class_='chart-list-item__ministats-cell-heading').text == 'Weeks':
                    songWeeks = c.text
                    songWeeksCleaned = songWeeks.strip().split('W')[0].strip()
            # appending to the data list
            billboard_info_list.append(
                {'date': this_date, 'song': list_title, 'performer': list_artist, 'chart_position': list_rank, 
                'previous_position': lastWeekCleaned, 'peak': songPeakCleaned, 'weeks_on_chart': songWeeksCleaned,
                'hitTF': 1})  
    # clicking on the dropdown to get to the next week link
    for l in span_section.find('label').find('ul').find_all('a'):
        if 'Next Week' in l.text:
            partial_link = l['href']
            
    
    # open the menu to get the next week
    browser.find_by_css('span.chart-detail-header__date-selector').first.click()
    
    # ensuring that if it is the last week's data to be collected, it won't try finding 'next week' option
    if counter != num_weeks_since - 1:
        # clicking on the link to the next week
        try:
            print(partial_link)
            browser.click_link_by_partial_href(partial_link)
            sleep(randint(3,10))
        except AttributeError as e:
            print(e)
    
#     increment the counter
    counter +=1


May 8, 2021


/charts/Hot-100/2021-05-15


/Users/sarahdalleyhood/~Applications/lib/python3.8/site-packages/splinter/driver/webdriver/__init__.py:482: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  warnings.warn(



May 15, 2021




In [7]:
# examine the list
billboard_info_list

[{'date': '\nMay 8, 2021\n\n',
  'song': 'Save Your Tears',
  'performer': 'The Weeknd & Ariana Grande',
  'chart_position': '1',
  'previous_position': '6',
  'peak': '1',
  'weeks_on_chart': '20',
  'hitTF': 1},
 {'date': '\nMay 8, 2021\n\n',
  'song': 'Leave The Door Open',
  'performer': 'Silk Sonic (Bruno Mars & Anderson .Paak)',
  'chart_position': '2',
  'previous_position': '2',
  'peak': '1',
  'weeks_on_chart': '8',
  'hitTF': 1},
 {'date': '\nMay 8, 2021\n\n',
  'song': 'Peaches',
  'performer': 'Justin Bieber Featuring Daniel Caesar & Giveon',
  'chart_position': '3',
  'previous_position': '3',
  'peak': '1',
  'weeks_on_chart': '6',
  'hitTF': 1},
 {'date': '\nMay 8, 2021\n\n',
  'song': 'Rapstar',
  'performer': 'Polo G',
  'chart_position': '4',
  'previous_position': '1',
  'peak': '1',
  'weeks_on_chart': '3',
  'hitTF': 1},
 {'date': '\nMay 8, 2021\n\n',
  'song': 'Levitating',
  'performer': 'Dua Lipa Featuring DaBaby',
  'chart_position': '5',
  'previous_position'

In [8]:
browser.quit()

In [9]:
# examine the data as a df
bb21_df = pd.DataFrame(billboard_info_list)
bb21_df.head()

,date,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,"\nMay 8, 2021\n\n",Save Your Tears,The Weeknd & Ariana Grande,1,6,1,20,1
1,"\nMay 8, 2021\n\n",Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),2,2,1,8,1
2,"\nMay 8, 2021\n\n",Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,3,3,1,6,1
3,"\nMay 8, 2021\n\n",Rapstar,Polo G,4,1,1,3,1
4,"\nMay 8, 2021\n\n",Levitating,Dua Lipa Featuring DaBaby,5,5,5,30,1


In [10]:
bb21_df.shape

(200, 8)

In [11]:
# removing the /n from the dates
clean_dates = []
for date in bb21_df['date']:
    clean_date = date.strip()
    clean_dates.append(clean_date)
clean_dates[0:5]

['May 8, 2021', 'May 8, 2021', 'May 8, 2021', 'May 8, 2021', 'May 8, 2021']

In [12]:
# adding the cleaned dates to the df
# bb21_df = bb21_df[['rank', 'artist', 'title']]
bb21_df['date'] = clean_dates
bb21_df.head()

,date,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,"May 8, 2021",Save Your Tears,The Weeknd & Ariana Grande,1,6,1,20,1
1,"May 8, 2021",Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),2,2,1,8,1
2,"May 8, 2021",Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,3,3,1,6,1
3,"May 8, 2021",Rapstar,Polo G,4,1,1,3,1
4,"May 8, 2021",Levitating,Dua Lipa Featuring DaBaby,5,5,5,30,1


In [13]:
# change date col to datetime
bb21_df['date'] = pd.to_datetime(bb21_df['date'])
bb21_df.head()

,date,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,2021-05-08,Save Your Tears,The Weeknd & Ariana Grande,1,6,1,20,1
1,2021-05-08,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),2,2,1,8,1
2,2021-05-08,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,3,3,1,6,1
3,2021-05-08,Rapstar,Polo G,4,1,1,3,1
4,2021-05-08,Levitating,Dua Lipa Featuring DaBaby,5,5,5,30,1


In [30]:
bb21_df.columns

Index(['date', 'song', 'performer', 'chart_position', 'previous_position',
       'peak', 'weeks_on_chart', 'hitTF'],
      dtype='object')

In [31]:
# initial shaping of the data for add on: removing date column
freshData = bb21_df[['song', 'performer', 'chart_position', 'previous_position', 'peak', 'weeks_on_chart', 'hitTF']]
freshData.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,Save Your Tears,The Weeknd & Ariana Grande,1,6,1,20,1
1,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),2,2,1,8,1
2,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,3,3,1,6,1
3,Rapstar,Polo G,4,1,1,3,1
4,Levitating,Dua Lipa Featuring DaBaby,5,5,5,30,1


### Getting Spotify track info

In [43]:
# additional imports
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [44]:
# preparing spotify credentials
load_dotenv(dotenv_path='spotifyCred.env')
CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')

In [45]:
# connecting to spotipy
spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
)

In [46]:
# getting only the song titles
song_titles_bb = freshData[['song', 'performer']]
song_titles_bb.loc[:, 'performer'] = song_titles_bb.loc[:, 'performer'].apply(lambda x: x.split('Featuring')[0])
song_titles_bb.loc[:, 'performer'] = song_titles_bb.loc[:, 'performer'].apply(lambda x: x.split('&')[0])
song_titles_bb

/Users/sarahdalleyhood/~Applications/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,song,performer
0,Save Your Tears,The Weeknd
1,Leave The Door Open,Silk Sonic (Bruno Mars
2,Peaches,Justin Bieber
3,Rapstar,Polo G
4,Levitating,Dua Lipa
...,...,...
195,If Pain Was A Person,Moneybagg Yo
196,Tampa,Cico P
197,Hello,Pop Smoke
198,Way Less Sad,AJR


In [47]:
# function to get spotify uris for each 2021 song
def get_song_uri(df, col_with_song_name):
    song_list = {
        'artist': [],
        'song': [],
        'uri': []
    }

    for i, row in df.iterrows():
        song = row[f'{col_with_song_name}']
    #     artist_from_df = row['performer'].lower()
        results = spotify.search(q=song, limit=5)['tracks']['items']
        for track in results:
            song_list['artist'].append(track['artists'][0]['name'])
            song_list['song'].append(track['name'])
            song_list['uri'].append(track['uri'])
            
    return song_list

In [48]:
# applying uri funct to 2021
song_uri_df = pd.DataFrame(get_song_uri(song_titles_bb, 'song'))
# examining 2021 df w/ uris
song_uri_df.head()

,artist,song,uri
0,The Weeknd,Save Your Tears,spotify:track:5QO79kh1waicV47BqGRL3g
1,The Weeknd,Save Your Tears (with Ariana Grande) (Remix),spotify:track:37BZB0z9T8Xu7U3e65qxFy
2,Lofi Fruits Music,Save Your Tears,spotify:track:7aXgt6bprirdbZrrC0fsqx
3,The Weeknd,Save Your Tears,spotify:track:2ExKb6Ag2WXob6FpkSeXhE
4,Boyce Avenue,Save Your Tears,spotify:track:45SbQo5bq8f0hNdB70IyFC


In [49]:
# getting spotify song features for each song
features = {
    'song': [],
    'artist': [],
    'features': []
}
# let code run till a break
for i, row in song_uri_df.iterrows():
    uri = row['uri']
    audio_features = spotify.audio_features(uri)[0]
    features['song'].append(row['song'])
    features['artist'].append(row['artist'])
    features['features'].append(audio_features)

In [52]:
# making the results of the api call into a df
m = pd.DataFrame(features)
m.head()

,song,artist,features
0,Save Your Tears,The Weeknd,"{'danceability': 0.68, 'energy': 0.826, 'key':..."
1,Save Your Tears (with Ariana Grande) (Remix),The Weeknd,"{'danceability': 0.65, 'energy': 0.825, 'key':..."
2,Save Your Tears,Lofi Fruits Music,"{'danceability': 0.733, 'energy': 0.22, 'key':..."
3,Save Your Tears,The Weeknd,"{'danceability': 0.684, 'energy': 0.817, 'key'..."
4,Save Your Tears,Boyce Avenue,"{'danceability': 0.735, 'energy': 0.275, 'key'..."


In [53]:
# renaming features output, renaming cols to fit with other dfs
bb_features = m
bb_features = bb_features.rename(columns={'artist': 'performer'})
bb_features

,song,performer,features
0,Save Your Tears,The Weeknd,"{'danceability': 0.68, 'energy': 0.826, 'key':..."
1,Save Your Tears (with Ariana Grande) (Remix),The Weeknd,"{'danceability': 0.65, 'energy': 0.825, 'key':..."
2,Save Your Tears,Lofi Fruits Music,"{'danceability': 0.733, 'energy': 0.22, 'key':..."
3,Save Your Tears,The Weeknd,"{'danceability': 0.684, 'energy': 0.817, 'key'..."
4,Save Your Tears,Boyce Avenue,"{'danceability': 0.735, 'energy': 0.275, 'key'..."
...,...,...,...
994,THANKFUL (feat. Lil Wayne & Jeremih),DJ Khaled,"{'danceability': 0.352, 'energy': 0.758, 'key'..."
995,thankful,Meltycanon,"{'danceability': 0.488, 'energy': 0.0866, 'key..."
996,Miss Independent,Kelly Clarkson,"{'danceability': 0.655, 'energy': 0.623, 'key'..."
997,Thankful,LiKKma,"{'danceability': 0.0644, 'energy': 0.0435, 'ke..."


In [54]:
# expanding the features col into separate cols
feats = bb_features['features'].apply(pd.Series)

In [55]:
# recombining the features into the 2021 features df
bb_features_df = pd.concat([bb_features, feats], axis=1).drop('features', axis=1)
bb_features_df.head()

,song,performer,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Save Your Tears,The Weeknd,0.680,0.826,0,-5.487,1,0.0309,0.0212,0.000012,0.5430,0.644,118.051,audio_features,5QO79kh1waicV47BqGRL3g,spotify:track:5QO79kh1waicV47BqGRL3g,https://api.spotify.com/v1/tracks/5QO79kh1waic...,https://api.spotify.com/v1/audio-analysis/5QO7...,215627,4
1,Save Your Tears (with Ariana Grande) (Remix),The Weeknd,0.650,0.825,0,-4.645,1,0.0325,0.0215,0.000024,0.0936,0.593,118.091,audio_features,37BZB0z9T8Xu7U3e65qxFy,spotify:track:37BZB0z9T8Xu7U3e65qxFy,https://api.spotify.com/v1/tracks/37BZB0z9T8Xu...,https://api.spotify.com/v1/audio-analysis/37BZ...,191014,4
2,Save Your Tears,Lofi Fruits Music,0.733,0.220,9,-21.308,0,0.0354,0.8820,0.974000,0.1150,0.396,143.984,audio_features,7aXgt6bprirdbZrrC0fsqx,spotify:track:7aXgt6bprirdbZrrC0fsqx,https://api.spotify.com/v1/tracks/7aXgt6bprird...,https://api.spotify.com/v1/audio-analysis/7aXg...,120047,4
3,Save Your Tears,The Weeknd,0.684,0.817,0,-5.499,1,0.0308,0.0222,0.000022,0.4390,0.623,118.061,audio_features,2ExKb6Ag2WXob6FpkSeXhE,spotify:track:2ExKb6Ag2WXob6FpkSeXhE,https://api.spotify.com/v1/tracks/2ExKb6Ag2WXo...,https://api.spotify.com/v1/audio-analysis/2ExK...,215627,4
4,Save Your Tears,Boyce Avenue,0.735,0.275,8,-8.994,0,0.0304,0.7460,0.000000,0.1310,0.294,116.004,audio_features,45SbQo5bq8f0hNdB70IyFC,spotify:track:45SbQo5bq8f0hNdB70IyFC,https://api.spotify.com/v1/tracks/45SbQo5bq8f0...,https://api.spotify.com/v1/audio-analysis/45Sb...,216800,4


In [56]:
# merging the original 2021 df with the newly obtained track features
fresh_merge = pd.merge(bb_features_df, freshData)
fresh_merge.head()

,song,performer,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,uri,track_href,analysis_url,duration_ms,time_signature,chart_position,previous_position,peak,weeks_on_chart,hitTF
0,Astronaut In The Ocean,Masked Wolf,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.0000,...,spotify:track:3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4,8,7,7,11,1
1,Astronaut In The Ocean,Masked Wolf,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.0000,...,spotify:track:3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4,7,8,7,12,1
2,Astronaut In The Ocean,Masked Wolf,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.0000,...,spotify:track:3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4,8,7,7,11,1
3,Astronaut In The Ocean,Masked Wolf,0.778,0.695,4,-6.865,0,0.0913,0.1750,0.0000,...,spotify:track:3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4,7,8,7,12,1
4,Up,Cardi B,0.868,0.795,11,-6.044,1,0.2690,0.0012,0.0193,...,spotify:track:1XXimziG1uhM0eDNCZCrUl,https://api.spotify.com/v1/tracks/1XXimziG1uhM...,https://api.spotify.com/v1/audio-analysis/1XXi...,156945,4,9,9,1,12,1


In [59]:
# removing and moving around cols
fresh_merge = fresh_merge[['song', 'performer', 'chart_position',
       'previous_position', 'peak', 'weeks_on_chart', 'hitTF', 
       'id', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 
       'duration_ms', 'time_signature']]
fresh_merge.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Astronaut In The Ocean,Masked Wolf,8,7,7,11,1,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,...,-6.865,0,0.0913,0.1750,0.0000,0.1500,0.472,149.996,132780,4
1,Astronaut In The Ocean,Masked Wolf,7,8,7,12,1,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,...,-6.865,0,0.0913,0.1750,0.0000,0.1500,0.472,149.996,132780,4
2,Astronaut In The Ocean,Masked Wolf,8,7,7,11,1,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,...,-6.865,0,0.0913,0.1750,0.0000,0.1500,0.472,149.996,132780,4
3,Astronaut In The Ocean,Masked Wolf,7,8,7,12,1,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,...,-6.865,0,0.0913,0.1750,0.0000,0.1500,0.472,149.996,132780,4
4,Up,Cardi B,9,9,1,12,1,1XXimziG1uhM0eDNCZCrUl,0.868,0.795,...,-6.044,1,0.2690,0.0012,0.0193,0.0461,0.819,166.000,156945,4


In [ ]:
# replacing '-' in previous position cols with 0
fresh_merge['previous_position'] = fresh_merge['previous_position'].replace(['-'], 0)

In [62]:
# sorting for aid in removing duplicates
# using weeks on chart as proxy for date
fresh_merge_sorted = fresh_merge.sort_values(['song','weeks_on_chart'])
fresh_merge_sorted.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
48,34+35,Ariana Grande,27,26,2,26,1,6Im9k8u9iIzKMrmV7BWtlF,0.83,0.585,...,-6.476,1,0.094,0.237,0.0,0.248,0.485,109.978,173711,4
50,34+35,Ariana Grande,27,26,2,26,1,6Im9k8u9iIzKMrmV7BWtlF,0.83,0.585,...,-6.476,1,0.094,0.237,0.0,0.248,0.485,109.978,173711,4
52,34+35,Ariana Grande,27,26,2,26,1,6Im9k8u9iIzKMrmV7BWtlF,0.83,0.585,...,-6.476,1,0.094,0.237,0.0,0.248,0.485,109.978,173711,4
54,34+35,Ariana Grande,27,26,2,26,1,6Im9k8u9iIzKMrmV7BWtlF,0.83,0.585,...,-6.476,1,0.094,0.237,0.0,0.248,0.485,109.978,173711,4
49,34+35,Ariana Grande,29,27,2,27,1,6Im9k8u9iIzKMrmV7BWtlF,0.83,0.585,...,-6.476,1,0.094,0.237,0.0,0.248,0.485,109.978,173711,4


In [63]:
# removing true duplicates (every bit of info the same)
fresh_merge_drops = fresh_merge_sorted.drop_duplicates(subset=['song', 'weeks_on_chart'], keep='first')
fresh_merge_drops.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
48,34+35,Ariana Grande,27,26,2,26,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1,0.0940,0.237,0.000000,0.248,0.485,109.978,173711,4
49,34+35,Ariana Grande,29,27,2,27,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1,0.0940,0.237,0.000000,0.248,0.485,109.978,173711,4
249,Almost Maybes,Jordan Davis,77,76,73,14,1,14JFKtBSqefxfMiKhq6JT8,0.582,0.868,...,-3.927,1,0.0522,0.146,0.000000,0.103,0.555,90.105,179153,4
250,Almost Maybes,Jordan Davis,71,77,71,15,1,14JFKtBSqefxfMiKhq6JT8,0.582,0.868,...,-3.927,1,0.0522,0.146,0.000000,0.103,0.555,90.105,179153,4
302,Anyone,Justin Bieber,87,68,6,17,1,31qCy5ZaophVA81wtlwLc4,0.686,0.538,...,-8.026,1,0.0345,0.181,0.000003,0.113,0.584,115.884,190779,4


In [64]:
# removing track duplicates (same song on different dates)
fresh_merge_doubledrops = fresh_merge_drops.drop_duplicates(subset=['song', 'performer'], keep='last')
fresh_merge_doubledrops.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
49,34+35,Ariana Grande,29,27,2,27,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1,0.0940,0.237,0.000000,0.248,0.485,109.978,173711,4
250,Almost Maybes,Jordan Davis,71,77,71,15,1,14JFKtBSqefxfMiKhq6JT8,0.582,0.868,...,-3.927,1,0.0522,0.146,0.000000,0.103,0.555,90.105,179153,4
302,Anyone,Justin Bieber,87,68,6,17,1,31qCy5ZaophVA81wtlwLc4,0.686,0.538,...,-8.026,1,0.0345,0.181,0.000003,0.113,0.584,115.884,190779,4
308,Arcade,Duncan Laurence,89,90,89,4,1,1Xi84slp6FryDSCbzq4UCD,0.450,0.329,...,-12.603,0,0.0441,0.818,0.001090,0.135,0.266,71.884,183624,3
1,Astronaut In The Ocean,Masked Wolf,7,8,7,12,1,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,...,-6.865,0,0.0913,0.175,0.000000,0.150,0.472,149.996,132780,4


In [65]:
# resetting index
fresh_merge_doubledrops = fresh_merge_doubledrops.reset_index(drop=True)
fresh_merge_doubledrops.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,34+35,Ariana Grande,29,27,2,27,1,6Im9k8u9iIzKMrmV7BWtlF,0.830,0.585,...,-6.476,1,0.0940,0.237,0.000000,0.248,0.485,109.978,173711,4
1,Almost Maybes,Jordan Davis,71,77,71,15,1,14JFKtBSqefxfMiKhq6JT8,0.582,0.868,...,-3.927,1,0.0522,0.146,0.000000,0.103,0.555,90.105,179153,4
2,Anyone,Justin Bieber,87,68,6,17,1,31qCy5ZaophVA81wtlwLc4,0.686,0.538,...,-8.026,1,0.0345,0.181,0.000003,0.113,0.584,115.884,190779,4
3,Arcade,Duncan Laurence,89,90,89,4,1,1Xi84slp6FryDSCbzq4UCD,0.450,0.329,...,-12.603,0,0.0441,0.818,0.001090,0.135,0.266,71.884,183624,3
4,Astronaut In The Ocean,Masked Wolf,7,8,7,12,1,3Ofmpyhv5UAQ70mENzB277,0.778,0.695,...,-6.865,0,0.0913,0.175,0.000000,0.150,0.472,149.996,132780,4


In [72]:
# getting last date recorded in the scrape
last_date_scraped = str(bb21_df['date'][len(bb21_df['date']) - 1]).split(' ')[0]
scrape_record_dict = {'last_date_scraped' : last_date_scraped}

# here is where the data would be sent off to sql, but for now, sent to csv


pd.DataFrame(scrape_record_dict, index=[0]).to_csv('../data/scrapeRecord.csv', index=False)
scrape_record_dict

{'last_date_scraped': '2021-05-15'}

In [29]:
# reading in the non-updated data
dataToUpdate = pd.read_csv('../data/billboardAndFeatures2018to2021.csv')
# in future:
# dataToUpdate = pd.read_csv('../data/fullDataUPDATE.csv')
dataToUpdate.head()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Dance Monkey,Tones And I,11.0,19.0,11.0,8.0,1.0,2XU0oxnq2qxCpomAAuJY8K,0.824,0.588,...,-6.400,0.0,0.0924,0.6920,0.000104,0.149,0.513,98.027,209438.0,4.0
1,Mine,Bazzi,56.0,0.0,56.0,1.0,1.0,7uzmGiiJyRfuViKKK3lVmR,0.710,0.789,...,-3.874,1.0,0.0722,0.0161,0.000003,0.451,0.717,142.929,131064.0,4.0
2,Final Fantasy,Drake,56.0,0.0,56.0,1.0,1.0,44Du2IM1bGY7dicmLfXbUs,0.500,0.449,...,-10.977,1.0,0.4420,0.4220,0.000008,0.115,0.104,144.206,219960.0,1.0
3,Hear Me Calling,Juice WRLD,56.0,0.0,56.0,1.0,1.0,13ZyrkCDmRz5xY3seuAWYk,0.699,0.687,...,-3.997,0.0,0.1060,0.3080,0.000036,0.121,0.499,88.932,189977.0,4.0
4,Liar,Camila Cabello,56.0,0.0,56.0,1.0,1.0,7LzouaWGFCy4tkXDOOnEyM,0.740,0.498,...,-6.684,0.0,0.0456,0.0169,0.002820,0.319,0.652,98.016,207039.0,4.0


In [77]:
# adding the newly scraped data to the rest of the data
dataUpdated = pd.concat([dataToUpdate, fresh_merge_doubledrops])
dataUpdated.tail()

,song,performer,chart_position,previous_position,peak,weeks_on_chart,hitTF,id,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
50,What You Know Bout Love,Pop Smoke,26,19,9,35,1.0,1tkg4EHVoqnhR6iFEXb60y,0.709,0.548,...,-8.493,1.0,0.3530,0.6500,0.000002,0.1330,0.5430,83.995,160000.0,4.0
51,What's Next,Drake,45,30,1,9,1.0,3mDFLytDotXo2p0rvfGbkA,0.777,0.580,...,-6.928,0.0,0.0525,0.0125,0.000000,0.1610,0.0636,129.918,178154.0,4.0
52,Wockesha,Moneybagg Yo,57,59,57,2,1.0,0nqhKXDjsyBMvbeWmgijD0,0.582,0.537,...,-9.229,1.0,0.6190,0.1290,0.000000,0.0749,0.7710,82.103,180930.0,4.0
53,Your Power,Billie Eilish,10,-,10,1,1.0,5qNh5WtzMbfpSj2jLlBkoD,0.634,0.285,...,-14.007,0.0,0.0807,0.9370,0.000359,0.2320,0.2030,129.650,245897.0,4.0
54,pov,Ariana Grande,55,66,40,9,1.0,3UoULw70kMsiVXxW0L3A33,0.487,0.534,...,-5.664,0.0,0.0555,0.3600,0.000000,0.1000,0.1730,131.798,201882.0,4.0


In [78]:
# exporting the renewed data to csv
dataUpdated.to_csv('../data/fullDataUPDATE.csv', index=False)